In [334]:
# %%writefile Generate_Base_Data.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
import warnings

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

%load_ext autoreload
%autoreload 2
%aimport tools.Cacher,tools.Sample_Tools


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [230]:
def cache_stock_qfq(codes,self_obj=None):
    import tools.Sample_Tools as smpl
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    import warnings
    
    for code in codes:
        try:
            stock_df = smpl.get_data([code], start='2000-01-01', end='2023-01-01').to_qfq().data
            dt_index = stock_df.index.get_level_values(0)
        except:
            # warnings.warn('get "{}" error'.format(code))
            continue

        try:
            save_cache('{}_train_qfq'.format(code) ,stock_df.loc[(dt_index >= '2017') & (dt_index < '2022')], cache_type=CACHE_TYPE.STOCK)
            save_cache('{}_tail_qfq'.format(code), stock_df.loc[(dt_index >= '2022')], cache_type=CACHE_TYPE.STOCK)
        except:
            warnings.warn('1.save_cache "{}" error'.format(code))
            continue

        try:
            save_cache('{}_older_qfq'.format(code), stock_df.loc[dt_index < '2017'],cache_type=CACHE_TYPE.STOCK)
        except:
            warnings.warn('2.save_cache "{}" error'.format(code))
            continue
        # print('{} saved'.format(code))
    

def generate_stock_qfq(,worker=6):
    codes = smpl.get_codes_from_blockname('沪深300', sse='all')
    task = task_chunk_split(codes, worker)
    results = parallal_task(worker, cache_stock_qfq, task)

In [354]:
# l = smpl.get_codes_from_blockname('沪深300', sse='all')
# def assemble_stocks_by_codes(codes):
#     files = list(map(lambda x:x+'_train_qfq',l))
#     return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

# df_all =  assemble_stocks_by_codes(l)


In [ ]:
df_all

open        high         low       close    volume  \
date       code                                                               
2017-01-03 000001    8.381056    8.445455    8.362656    8.427055  459840.0   
           000002   15.787499   16.041021   15.787499   15.925784  217016.0   
           000063   15.613349   15.886753   15.525469   15.818402  200629.0   
           000066   12.121133   12.189065   12.024086   12.121133   74610.0   
           000069    5.640928    5.657091    5.600520    5.632846  362413.0   
...                       ...         ...         ...         ...       ...   
2021-12-31 603882  107.250391  110.303432  106.856450  109.682975   37879.0   
           603899   64.071489   64.308426   62.689362   63.686468   31908.0   
           603986  176.946440  176.946440  170.901316  174.554199  112160.0   
           603993    5.474964    5.524288    5.455235    5.504559  862563.0   
           605499  177.028490  179.876769  175.554901  179.837210    7625.0   

                         amount       adj  
date       code                            
2017-01-03 000001  4.205952e+08  0.919984  
           000002  4.497575e+08  0.768248  
           000063  3.241164e+08  0.976445  
           000066  9.303146e+07  0.970467  
           000069  2.525984e+08  0.808156  
...                         ...       ...  
2021-12-31 603882  4.200908e+08  0.984852  
           603899  2.056899e+08  0.987234  
           603986  1.964312e+09  0.992631  
           603993  4.801268e+08  0.986480  
           605499  1.374758e+08  0.988986  

[276966 rows x 7 columns]

In [ ]:
df_all

open        high         low       close    volume  \
date       code                                                               
2017-01-03 000001    8.381056    8.445455    8.362656    8.427055  459840.0   
           000002   15.787499   16.041021   15.787499   15.925784  217016.0   
           000063   15.613349   15.886753   15.525469   15.818402  200629.0   
           000066   12.121133   12.189065   12.024086   12.121133   74610.0   
           000069    5.640928    5.657091    5.600520    5.632846  362413.0   
...                       ...         ...         ...         ...       ...   
2021-12-31 603882  107.250391  110.303432  106.856450  109.682975   37879.0   
           603899   64.071489   64.308426   62.689362   63.686468   31908.0   
           603986  176.946440  176.946440  170.901316  174.554199  112160.0   
           603993    5.474964    5.524288    5.455235    5.504559  862563.0   
           605499  177.028490  179.876769  175.554901  179.837210    7625.0   

                         amount       adj  
date       code                            
2017-01-03 000001  4.205952e+08  0.919984  
           000002  4.497575e+08  0.768248  
           000063  3.241164e+08  0.976445  
           000066  9.303146e+07  0.970467  
           000069  2.525984e+08  0.808156  
...                         ...       ...  
2021-12-31 603882  4.200908e+08  0.984852  
           603899  2.056899e+08  0.987234  
           603986  1.964312e+09  0.992631  
           603993  4.801268e+08  0.986480  
           605499  1.374758e+08  0.988986  

[276966 rows x 7 columns]